# Método Simplex de Duas Fases

#### Este notebook mostra o processo de desenvolvimento e aplicação do método simplex para resolução de problemas de programação linear, aos quais o objetivo seja minimizar ou maximizar o valor de uma função objetivo. Este algoritmo foi atualizado para ser capaz de resolver problemas cuja origem não faz parte das solução, o que é conhecido como método simplex de duas fases.

## Algoritmo Método Simplex de Duas Fases

#### O Código abaixo mostra o processo de codificação do método Simplex de duas fases. Sendo que antes de cada célula há uma breve explicação a respeito do conteúdo subsequente.

Importa-se a biblioteca *numpy* que será usada para criação e manipulação das matrizes

In [1]:
import numpy as np
import math

Neste momento, criaremos a função que gera o *tableoux*, que é a matriz que contém as restrições aplicáveis ao problema

In [2]:
class Solucao(object):
    def __init__(self, matrix):
        self.linhas = len(matrix)
        self.colunas = len(matrix[0])

    #verifica se vetor é canonico (compara com vetor canonico)
    def canonico(self, vector):
        temp = np.array(vector)
        temp = temp.transpose()

        base = np.eye(self.linhas)

        for i in range(self.linhas):
            if np.array_equal(temp, base[i,:]):
                return True

        return False

    #verifica solucao, caso alguma coluna da matriz tenha todos elementos menores ou iguais a zero significa que nao possui solucao
    def sem_solucao(self, matrix):
        for i in range(1, self.colunas):
            if matrix[0][i] > 0:
                if all(j <= 0 for j in matrix[1:,i]):
                    print("Tableau não possui solução única viável.")
                    print("Encontrada coluna com todos elementos menores ou iguais a zero")
                    print("Z = -inf")
                    print("Abortando")
                    return True

        return False

    #Verifica se se ja é solução otima,
    #ou seja, se todos os zj - cj são menores ou iguais a zero
    def solucao_otima(self, matrix):
        if all(i <= 0 for i in matrix[0,1:]):
            return True
        else:
            return False

    #Verifica se existe alguma variável não básica cujo zj-cj = 0
    #se houver, significa que há soluções infinitas
    def multipla_solucao(self, matrix):
        for i in range(1, self.colunas):
            if (not self.canonico(matrix[:,i])) and matrix[0][i] == 0:
                return True

        return False

    #Verifica no quadro ótimo existe alguma variável básica igual a zero
    #significa que solução é degenerada
    def degenerada(self, matrix):
        for i in range(1, self.colunas):
            if self.canonico(matrix[:,i]):
                posicao, = np.unravel_index(matrix[:,i].argmax(), matrix[:,i].shape)

                if matrix[posicao][0] == 0:
                    return True

        return False

    #monta os vetores X* e Z*
    def monta_solucao(self, matrix):
        x = np.zeros(self.colunas -1)

        for i in range(1, self.colunas):
            if self.canonico(matrix[:,i]):
                posicao, = np.unravel_index(matrix[:,i].argmax(), matrix[:,i].shape)
                x[i-1] = matrix[posicao][0]

        z = matrix[0][0]

        return x, z

    #verifica se array é de variaveis artificiais (nesse caso matrix é i por 1)
    def artificial(self, matrix):
        bases = np.eye(self.linhas - 1, dtype=int)

        for i in range(self.linhas - 1):
            #verificar se da segunda linha em diante forma uma base
            if np.array_equal(bases[i,:], matrix[1:]):
                if matrix[0] == -1:
                    return True

        return False

In [3]:
class MetodoSimplex(object):
    def __init__(self, matrix):
        self.s = Solucao(matrix)

    #verifica se primeira fase chegou ao fim
    def fim_primeira_fase(self, matrix, zArtif):
        if all(i <= 0 for i in matrix[0,1:]):
            if np.array_equal(matrix[0,:],zArtif):
                return True

    #calcula a primeira fase
    def primeira_fase(self, matrix, vb):
        print("\nIniciando primeira fase:")
        print("")

        print("Tableau inicial:")
        print(np.matrix(matrix))
        print("")

        numLinhas = len(matrix) - 2
        numColunas = len(matrix[0]) - 1

        zArt = matrix[0]
        count = 0
        passo = 0

        for i in range(numColunas + 1):
            if zArt[i] < 0:
                count += 1

        for i in range(1, numColunas + 1):
            if self.s.artificial(matrix[:,i].transpose()):
                posicao, = np.unravel_index(matrix[:, i].argmax(), matrix[:, i].shape)
                matrix[0,:] = matrix[0,:] + matrix[posicao,:]

        print("Quadro Tableau")
        print(np.matrix(matrix))
        print("")

        while not self.fim_primeira_fase(matrix, zArt):
            posicaoMaior, = np.unravel_index(matrix[0, 1:].argmax(), matrix[0, 1:].shape)
            posicaoMaior = posicaoMaior + 1 #precisa incrementar, visto que retorna a posicao relativa ao slice

            #cria um vetor para armazenar divisoes
            div = np.zeros(numLinhas)
            for i in range(numLinhas):
                div[i] = float("inf")


            for i in range(2, numLinhas + 2):
                if matrix[i][posicaoMaior] > 0:
                    div[i-2] = matrix[i][0] / matrix[i][posicaoMaior]

            #Procura menor divisao e guarda linha
            posicaoMenor, = np.unravel_index(div.argmin(), div.shape)
            posicaoMenor = posicaoMenor + 2

            pivo = matrix[posicaoMenor][posicaoMaior]

            if pivo == 0:
#                 exit(1)
                quit()

            if pivo != 1:
                matrix[posicaoMenor, :] = matrix[posicaoMenor, :] / pivo

            for i in range(len(matrix)):
                if i != posicaoMenor:
                    if matrix[i][posicaoMaior] != 0:
                        matrix[i, :] = matrix[i, :] - matrix[i][posicaoMaior] * matrix[posicaoMenor, :]

            passo = passo + 1
            print("\nTableau - Primeira fase - Passo: ", passo)
            print(matrix)

        #confere se e possivel continuar para segunda fase
        if matrix[0][0] != 0:
            print("Conjunto de soluções é vazio\n")
            print("Za:", matrix[0][0])
            print("Fim\n")
#             exit()
            quit()

        return matrix, count, vb

    #realiza segunda fase
    def simplex(self, matrix, vb):
        print("Tableau inicial:")
        print(matrix)
        print("")

        numLinhas = len(matrix)
        numColunas = len(matrix[0])
        numVB = numLinhas
        passo = 0

        if self.s.sem_solucao(matrix):
#             exit(1)
            quit()

        if self.s.solucao_otima(matrix):

            x,z = self.s.monta_solucao(matrix)

            print("Quadro ótimo:")
            print(matrix)
            print("X* = ", x)
            print("Z* = ", z)

            return matrix, x, z

        while not self.s.solucao_otima(matrix):
            posicaoMaior, = np.unravel_index(matrix[0, 1:].argmax(), matrix[0, 1:].shape)
            posicaoMaior = posicaoMaior + 1 #precisa incrementar, visto que retorna a posicao relativa ao slice

            #cria um vetor para armazenar divisoes
            div = np.zeros(numLinhas)
            for i in range(numLinhas):
                div[i] = float("inf")

            for i in range(1, numLinhas):
                if matrix[i][posicaoMaior] > 0:
                    div[i-1] = matrix[i][0] / matrix[i][posicaoMaior]

            #Procura o menor da divisao e guarda linhas
            #Procura menor divisao e guarda linha
            posicaoMenor, = np.unravel_index(div.argmin(), div.shape)
            posicaoMenor = posicaoMenor + 1

            pivo = matrix[posicaoMenor][posicaoMaior]

            vb[posicaoMenor] = posicaoMaior

            if pivo != 1:
                matrix[posicaoMenor, :] = matrix[posicaoMenor, :] / pivo


            for i in range(numLinhas):
                if i != posicaoMenor:
                    if matrix[i][posicaoMaior] != 0:
                        #calcula todas as linhas
                        matrix[i, :] = matrix[i, :] - matrix[i][posicaoMaior] * matrix[posicaoMenor, :]

            if self.s.sem_solucao(matrix):
#                 exit(1)
                quit()

            if self.s.solucao_otima(matrix):

                x,z = self.s.monta_solucao(matrix)

                print("\nQuadro ótimo:")
                print((matrix), "\n")
                print("X* = ", x)
                print("Z* = ", z)
            else:
                passo = passo + 1
                print("Tableu do passo: ", passo)
                print(np.matrix(matrix))
                print("")

        return matrix, x, z

In [4]:
matrix = np.array(
    [[1,0,0,0,0,0,1,1],
    [-1,-4,-1,0,0,0,0,0],
    [3,3,1,0,0,0,1,0],
    [6,4,3,0,-1,0,0,1],
    [4,1,2,0,0,1,0,0]])
vb = np.zeros(len(matrix))

simplex = MetodoSimplex(matrix)

In [5]:
#executa duas fases
matrix[0,:] = (-1) * matrix[0,:]
matrix[1,:] = (-1) * matrix[1,:]
matrix, count, vb = simplex.primeira_fase(matrix, vb)

print('\nFim da primeira fase\n')
print("Za:", matrix[0][0], "\n")
print("Iniciando Segunda fase:")

#Elimina linhas e colunas das variaveis artificiais
matrix = matrix[1:,:-count]
matrix[0,:] = (-1) * matrix[0,:]
simplex = MetodoSimplex(matrix)
matrix, x, z = simplex.simplex(matrix, vb)


# #executa apenas uma fase
# print("Iniciando simplex...")
# matrix[0,:] = (-1) * matrix[0,:]
# matrix, x, z = simplex.simplex(matrix, vb)



Iniciando primeira fase:

Tableau inicial:
[[-1  0  0  0  0  0 -1 -1]
 [ 1  4  1  0  0  0  0  0]
 [ 3  3  1  0  0  0  1  0]
 [ 6  4  3  0 -1  0  0  1]
 [ 4  1  2  0  0  1  0  0]]

Quadro Tableau
[[ 8  7  4  0 -1  0  0  0]
 [ 1  4  1  0  0  0  0  0]
 [ 3  3  1  0  0  0  1  0]
 [ 6  4  3  0 -1  0  0  1]
 [ 4  1  2  0  0  1  0  0]]


Tableau - Primeira fase - Passo:  1
[[ 1  0  4  0 -1  0  0  0]
 [-3  0  1  0  0  0  0  0]
 [ 1  1  0  0  0  0  0  0]
 [ 2  0  3  0 -1  0  0  1]
 [ 3  0  2  0  0  1  0  0]]

Tableau - Primeira fase - Passo:  2
[[ 1  0  0  0 -1  0  0  0]
 [-3  0  0  0  0  0  0  0]
 [ 1  1  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0]
 [ 3  0  0  0  0  1  0  0]]
Conjunto de soluções é vazio

Za: 1
Fim


Fim da primeira fase

Za: 1 

Iniciando Segunda fase:
Tableau inicial:
[[3 0 0 0 0]
 [1 1 0 0 0]
 [0 0 1 0 0]
 [3 0 0 0 0]]

Quadro ótimo:
[[3 0 0 0 0]
 [1 1 0 0 0]
 [0 0 1 0 0]
 [3 0 0 0 0]]
X* =  [1. 0. 0. 0.]
Z* =  3


In [6]:
matrix = np.array(
    [[1,0,0,0,0,0,0,0,1,1,1],
    [1,3,2,3,0,0,0,0,0,0,0],
    [2,2,1,1,0,0,0,0,1,0,0],
    [6,1,3,1,0,0,0,0,0,1,0],
    [8,3,4,2,0,0,0,0,0,0,1]])
vb = np.zeros(len(matrix))

simplex = MetodoSimplex(matrix)

In [7]:
#executa duas fases
matrix[0,:] = (-1) * matrix[0,:]
matrix[1,:] = (-1) * matrix[1,:]
matrix, count, vb = simplex.primeira_fase(matrix, vb)

print('\nFim da primeira fase\n')
print("Za:", matrix[0][0], "\n")
print("Iniciando Segunda fase:")

#Elimina linhas e colunas das variaveis artificiais
matrix = matrix[1:,:-count]
simplex = MetodoSimplex(matrix)
matrix, x, z = simplex.simplex(matrix, vb)


Iniciando primeira fase:

Tableau inicial:
[[-1  0  0  0  0  0  0  0 -1 -1 -1]
 [-1 -3 -2 -3  0  0  0  0  0  0  0]
 [ 2  2  1  1  0  0  0  0  1  0  0]
 [ 6  1  3  1  0  0  0  0  0  1  0]
 [ 8  3  4  2  0  0  0  0  0  0  1]]

Quadro Tableau
[[15  6  8  4  0  0  0  0  0  0  0]
 [-1 -3 -2 -3  0  0  0  0  0  0  0]
 [ 2  2  1  1  0  0  0  0  1  0  0]
 [ 6  1  3  1  0  0  0  0  0  1  0]
 [ 8  3  4  2  0  0  0  0  0  0  1]]


Tableau - Primeira fase - Passo:  1
[[ -1 -10   0  -4   0   0   0   0  -8   0   0]
 [  3   1   0  -1   0   0   0   0   2   0   0]
 [  2   2   1   1   0   0   0   0   1   0   0]
 [  0  -5   0  -2   0   0   0   0  -3   1   0]
 [  0  -5   0  -2   0   0   0   0  -4   0   1]]
Conjunto de soluções é vazio

Za: -1
Fim


Fim da primeira fase

Za: -1 

Iniciando Segunda fase:
Tableau inicial:
[[ 3  1  0 -1  0  0  0]
 [ 2  2  1  1  0  0  0]
 [ 0 -5  0 -2  0  0  0]
 [ 0 -5  0 -2  0  0  0]]


Quadro ótimo:
[[ 2  0  0 -1  0  0  0]
 [ 1  1  0  0  0  0  0]
 [ 5  0  0 -2  0  0  0]
 [ 5

In [8]:
matrix = np.array(
    [[1,0,0,0,0,0,1,1],
    [1,3,2,3,0,0,0,0],
    [2,2,1,1,0,0,1,0],
    [8,3,4,2,0,0,0,1],
    [0,0,0,0,0,0,0,0]])
vb = np.zeros(len(matrix))

simplex = MetodoSimplex(matrix)

In [9]:
#executa duas fases
matrix[0,:] = (-1) * matrix[0,:]
matrix[1,:] = (-1) * matrix[1,:]
matrix, count, vb = simplex.primeira_fase(matrix, vb)

print('\nFim da primeira fase\n')
print("Za:", matrix[0][0], "\n")
print("Iniciando Segunda fase:")

#Elimina linhas e colunas das variaveis artificiais
matrix = matrix[1:,:-count]
simplex = MetodoSimplex(matrix)
matrix, x, z = simplex.simplex(matrix, vb)


Iniciando primeira fase:

Tableau inicial:
[[-1  0  0  0  0  0 -1 -1]
 [-1 -3 -2 -3  0  0  0  0]
 [ 2  2  1  1  0  0  1  0]
 [ 8  3  4  2  0  0  0  1]
 [ 0  0  0  0  0  0  0  0]]

Quadro Tableau
[[ 9  5  5  3  0  0  0  0]
 [-1 -3 -2 -3  0  0  0  0]
 [ 2  2  1  1  0  0  1  0]
 [ 8  3  4  2  0  0  0  1]
 [ 0  0  0  0  0  0  0  0]]


Tableau - Primeira fase - Passo:  1
[[ 4  0  5  3  0  0  0  0]
 [ 2  0 -2 -3  0  0  0  0]
 [ 1  1  0  0  0  0  0  0]
 [ 5  0  4  2  0  0  0  1]
 [ 0  0  0  0  0  0  0  0]]

Tableau - Primeira fase - Passo:  2
[[-1  0  0  3  0  0  0  0]
 [ 4  0  0 -3  0  0  0  0]
 [ 1  1  0  0  0  0  0  0]
 [ 1  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]

Tableau - Primeira fase - Passo:  3
[[ 9223372036854775807 -9223372036854775808 -9223372036854775808
  -9223372036854775805 -9223372036854775808 -9223372036854775808
  -9223372036854775808 -9223372036854775808]
 [-9223372036854775804 -9223372036854775808 -9223372036854775808
   9223372036854775805 -9223372036854775808 

<ipython-input-3-651432093859>:65: RuntimeWarning: divide by zero encountered in true_divide
  matrix[posicaoMenor, :] = matrix[posicaoMenor, :] / pivo
<ipython-input-3-651432093859>:65: RuntimeWarning: invalid value encountered in true_divide
  matrix[posicaoMenor, :] = matrix[posicaoMenor, :] / pivo


In [10]:
matrix = np.array(
    [[1,0,0,0,0,0,0,1,1,1],
    [1,2,-4,3,0,0,0,0,0,0],
    [5,5,-6,2,-1,0,0,0,0,1],
    [8,-1,3,5,0,-1,0,0,1,0],
    [4,2,5,-4,0,0,1,0,0,1]])
vb = np.zeros(len(matrix))

simplex = MetodoSimplex(matrix)

In [11]:
#executa duas fases
matrix[0,:] = (-1) * matrix[0,:]
matrix[1,:] = (-1) * matrix[1,:]
matrix, count, vb = simplex.primeira_fase(matrix, vb)

print('\nFim da primeira fase\n')
print("Za:", matrix[0][0], "\n")
print("Iniciando Segunda fase:")

#Elimina linhas e colunas das variaveis artificiais
matrix = matrix[1:,:-count]
simplex = MetodoSimplex(matrix)
matrix, x, z = simplex.simplex(matrix, vb)


Iniciando primeira fase:

Tableau inicial:
[[-1  0  0  0  0  0  0 -1 -1 -1]
 [-1 -2  4 -3  0  0  0  0  0  0]
 [ 5  5 -6  2 -1  0  0  0  0  1]
 [ 8 -1  3  5  0 -1  0  0  1  0]
 [ 4  2  5 -4  0  0  1  0  0  1]]

Quadro Tableau
[[ 7 -1  3  5  0 -1  0 -1  0 -1]
 [-1 -2  4 -3  0  0  0  0  0  0]
 [ 5  5 -6  2 -1  0  0  0  0  1]
 [ 8 -1  3  5  0 -1  0  0  1  0]
 [ 4  2  5 -4  0  0  1  0  0  1]]


Tableau - Primeira fase - Passo:  1
[[ 2 -1  3  0  0 -1  0 -1  0 -1]
 [ 2 -2  4  0  0  0  0  0  0  0]
 [ 3  5 -6  0 -1  0  0  0  0  1]
 [ 1  0  0  1  0  0  0  0  0  0]
 [ 8  2  5  0  0  0  1  0  0  1]]

Tableau - Primeira fase - Passo:  2
[[-1 -1  0  0  0 -1  0 -1  0 -1]
 [-2 -2  0  0  0  0  0  0  0  0]
 [ 9  5  0  0 -1  0  0  0  0  1]
 [ 1  0  0  1  0  0  0  0  0  0]
 [ 1  0  1  0  0  0  0  0  0  0]]
Conjunto de soluções é vazio

Za: -1
Fim


Fim da primeira fase

Za: -1 

Iniciando Segunda fase:
Tableau inicial:
[[-2 -2  0  0  0  0]
 [ 9  5  0  0 -1  0]
 [ 1  0  0  1  0  0]
 [ 1  0  1  0  0  0]]

